# Mod 2 Assessment

### Congratulations on making it to your second assessment! Just a few reminders before you continue:
- This should only take an hour, so be sure to manage your time effectively.
- Read the instructions carefully for _specified variable names_.
- Check your progress by running **`make tests`** in the terminal, </br>
  OR **if you have a PC**, manually run these lines in the terminal:
  - `jupyter nbconvert --to script mod2_assessment.ipynb`
  - `python -m pytest --disable-pytest-warnings -v`

If there is any confusion on a question, please ask for clarification from a coach. </br>Though we can't give you the answer, we can help clear up any misunderstandings and get you back on track.

In [19]:
# You'll need these imports to start.
import pymongo
from bson.json_util import loads
from collections import Counter

# put all of your additional imports here:
import requests
import sqlite3
import json
import pandas as pd
from bs4 import BeautifulSoup

## Section 1: SQL (20 Minutes)
There is a sqlite3 database in `assets/books.db`. The SQL to create this is also in `assets/books.sql`; if you want to run it manually you can also import this using https://sqliteonline.com/, or run the SQL file directly.  Both have the same schema and data.

The schema has three tables. You can explore the schema in the file posted above.  Please answer the following questions.

Connect to the database using sqlite3. <br/>
Assign `conn` to the sqlite3 connection <br/>
Assign `cur`  to the connection's cursor object

In [20]:
conn = sqlite3.connect("books-Copy1.db")
cur = conn.cursor()

#### Querying the DB: 

1. How many pages are in the book "Nine Stories"?

In [21]:
# Assign `answer_1` to your final answer
cur.execute("""
SELECT title, pages FROM book
WHERE title LIKE "Nine Stories" """)

answer_1 = cur.fetchall()

2. How many authors are from the USA?

In [22]:
# Assign `answer_2` to your final answer
cur.execute("""
SELECT COUNT(author_id), country FROM author
WHERE country LIKE "USA" """)

answer_2 = cur.fetchall()

3. How many authors does the book "Professional ASP.NET 4.5 in C# and VB" have?

In [23]:
# Assign `answer_3` to your final answer
cur.execute("""
SELECT COUNT(author_id), title FROM book
JOIN book_author ON book.book_id = book_author.book_id 
WHERE title LIKE "Professional ASP.NET 4.5 in C# and VB" """)
answer_3 = cur.fetchall()

4. How many pages total have been written by non-American authors?

In [24]:
# Assign `answer_4` to your final answer
cur.execute("""
SELECT SUM(pages) FROM book
JOIN author
WHERE country NOT LIKE "USA" """)
answer_4 = cur.fetchall()

## Section 2: Object Oriented Programming (10 Minutes)

### Creating a Class
1. Force every new instance of `WeWorkMember` to expect a value which is assigned to the `name` **attribute**.
1. Give every new instance of `WeWorkMember` a `caffeinated` **attribute** that is set to `False`. 
1. Give `WeWorkMember` an **instance method** called `caffeinate()` that prints out "Getting coffee!" and sets the `caffeinated` attribute to `True`.

In [25]:
class WeWorkMember:
    # Constructor that initializes the name and caffeinated instance varaibles/attributes
    def __init__(self, name):
        self.name = name
        self.caffeinated = False
        
    # Instance method
    def caffeinate(self):
        print("Getting coffee!")
        self.caffeinated = True

### Inheriting from a Class

1. Have `Staff` and `Student` inherit all methods from `WeWorkMember`
1. Give `Staff` a **static method** called `cheer()` that prints out "Goooooooo Flatiron Students!"
1. Give `Students` a **class method** called `learn()` that takes in an integer and returns that number +1

In [26]:
class Staff(WeWorkMember):
    
    @staticmethod
    def cheer():
        print("Goooooooo Flatiron Students!")

class Student(WeWorkMember):
    
    @classmethod
    def learn(cls, num):
        cls.number = int(num)
        return cls.number + 1

In [27]:
st1 = Student("Jack")
st1.caffeinate()
print(st1.caffeinated)

Getting coffee!
True


In [28]:
st1.learn(45)

46

## Section 3: APIs & Web Scraping
### APIs (10 Minutes)
Using the API about RuPaul's Drag Race, tell me how many judges of each **`type`** there were in the **first 50 records** the API returns.

API: http://www.nokeynoshade.party/api/judges </br>
Docs: https://drag-race-api.readme.io/docs/get-all-judges

 - Assign `rupaul_resp` to the response of the API request.
 - Ensure the request only returns the first 50 records
     - note the documentation shows parameters as uppercase, but they should be **lowercase**
     - the judges will have **`id`** ranging from 1 to 53
 - Do the aggregation in pure Python, Pandas, or SQL -- whatever's easiest for you
 - Assign `judge_count` to a dictionary with the number of judges for each type

In [29]:
# Return the first 50 records using the id param
def records_50():
    record_limit = {"limit": 47}
    rupaul_resp = requests.get("http://www.nokeynoshade.party/api/judges", record_limit)
    return rupaul_resp.json()
    
records_50()

[{'id': 1,
  'name': 'Rupaul',
  'image_url': 'https://vignette.wikia.nocookie.net/logosrupaulsdragrace/images/b/ba/Rupaul_blackpink_final.jpg/revision/latest/scale-to-width-down/350?cb=20110731183922',
  'bio': 'Drag performer, actor, television host, and recording artist',
  'type': 'regular',
  'createdAt': '2018-01-11T00:43:31.258Z',
  'updatedAt': '2018-01-11T00:43:31.258Z'},
 {'id': 2,
  'name': 'Bob Mackie',
  'image_url': None,
  'bio': 'Fashion Designer',
  'type': 'guest',
  'createdAt': '2017-08-22T00:27:40.731Z',
  'updatedAt': '2017-08-22T00:27:40.731Z'},
 {'id': 3,
  'name': 'Mike Ruiz',
  'image_url': 'https://vignette3.wikia.nocookie.net/logosrupaulsdragrace/images/7/7c/Mike_Ruiz.jpg/revision/latest/scale-to-width-down/350?cb=20110731184513',
  'bio': 'Photographer',
  'type': 'regular',
  'createdAt': '2017-08-22T00:46:35.279Z',
  'updatedAt': '2017-08-22T00:46:35.279Z'},
 {'id': 4,
  'name': 'Michelle Williams',
  'image_url': 'https://upload.wikimedia.org/wikipedia/c

In [71]:
# Returning type and counting how many of each type their are
# We can use a for loop to iterate through the list and dict to get their type
judge_count = Counter([judge["type"] for judge in records_50()])

judge_count

Counter({'regular': 2, 'guest': 44, 'interim': 1})

### Web Scraping (15 Minutes)
We want all of the **Music** books from [http://books.toscrape.com/](http://books.toscrape.com/).  Download the page of books in the *music* category and save it to the `book_resp` variable. Then use BeautifulSoup to convert each book on the page into a row in a Pandas dataframe.  The dataframe should consist of five columns:
- FULL book title
- book cover image link (relative is fine)
- book cost (as a float or decimal, not a string)
- inventory status (in stock?)
- **BONUS** Star rating (if you choose not to include this, just fill the column with `NaN`s.

The final result should be a dataframe of 13 rows (not counting header) and 5 columns (not counting index), named `book_df`.

- Set the response to a variable named `book_resp`
- Set the BeautifulSoup() object to a variable named `book_soup`
- Set the final dataframe object to a variable named `book_df`

In [31]:
page = requests.get("http://books.toscrape.com/")
books_soup = BeautifulSoup(page.text, "html.parser")

In [32]:
# Scrapping the book titles
book_title = books_soup.find_all("h3", title="") # tag, attribute or class
# Looping through book title to get the elements
titles = []
for title in book_title:
    titles.append(title.find("a").get("title"))  # tag, attribute

# OR

title = [titles.find("a").get("title") for titles in book_title]

In [231]:
# Scrapping the book image link
book_image_link = books_soup.find_all("div", class_="image_container")
# Looping through to find and get the exact links
image_links = [links.find("img").get("src") for links in book_image_link]

In [251]:
book_cost = books_soup.find_all("p", class_="price_color")
cost = [price.text.split("Â")[1] for price in book_cost]

In [252]:
# Scrapping the inventory status of the books
inventory_status = books_soup.find_all("p", class_="instock availability")
availability = [stock.text.strip() for stock in inventory_status]

In [297]:
# Creating a DataFrame
book_info_df = pd.DataFrame([title, image_links, cost, availability]).T

# Creating column names
book_info_df.columns = ["title", "image_link", "cost", "availability"]

book_info_df

,title,image_link,cost,availability
0,A Light in the Attic,media/cache/2c/da/2cdad67c44b002e7ead0cc35693c...,£51.77,In stock
1,Tipping the Velvet,media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f...,£53.74,In stock
2,Soumission,media/cache/3e/ef/3eef99c9d9adef34639f51066202...,£50.10,In stock
3,Sharp Objects,media/cache/32/51/3251cf3a3412f53f339e42cac213...,£47.82,In stock
4,Sapiens: A Brief History of Humankind,media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c...,£54.23,In stock
5,The Requiem Red,media/cache/68/33/68339b4c9bc034267e1da611ab3b...,£22.65,In stock
6,The Dirty Little Secrets of Getting Your Dream...,media/cache/92/27/92274a95b7c251fea59a2b8a7827...,£33.34,In stock
7,The Coming Woman: A Novel Based on the Life of...,media/cache/3d/54/3d54940e57e662c4dd1f3ff00c78...,£17.93,In stock
8,The Boys in the Boat: Nine Americans and Their...,media/cache/66/88/66883b91f6804b2323c8369331cb...,£22.60,In stock
9,The Black Maria,media/cache/58/46/5846057e28022268153beff6d352...,£52.15,In stock


## Section 4: NoSQL (10 Minutes)

#### Load data from `assets/grades.json` into Mongo 

(this code is written for you)

In [7]:
# you shouldn't need to edit this cell!
db_name = "mod2db"

with open('Mod2Assessment/Data/grades.jsonl') as f:
    # loads() comes from the bson library
    file_data = [loads(line) for line in f.readlines()]

client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database(db_name)
db = client[db_name]
coll = db["testcoll"]

coll.insert_many(file_data)

#### Answer all of the following questions by querying Mongo and manipulating the results in Python

1. How many records are there total?

In [35]:
# Set `nosql_answer1` to your final answer
nosql_answer1 = coll.count_documents(filter = {})
nosql_answer1

280

2. How many students have taken the class with `class_id` = **29**?

In [34]:
# Set `nosql_answer2` to your final answer
nosql_answer2 = coll.count(filter = {"class_id": 29})
nosql_answer2

<ipython-input-34-697ea6239bad>:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  nosql_answer2 = coll.count(filter = {"class_id": 29})


9

**Super bonus question**: </br>For student **12** in class **23**, what grade did they get on their exam?

In [33]:
# Set `nosql_answer3` to your final answer
nosql_answer3 = coll.find_one(filter = {'class_id': 23, 'student_id': 12})['scores']
nosql_answer3[0]['score']

26.9857216299485

In [6]:
# for when you're done with this portion, this deletes the data we added.
client.drop_database("mod2db")

## Assessment submission (2 Minutes)
Please **save** your completed file (`mod2_assessment.ipynb`) and upload it using [this form](https://docs.google.com/forms/d/e/1FAIpQLSf1uGNuz4fyzVz5i3aFTxmMKvH50DEJiN5uRmNFghpmFzoi3g/viewform)